In [15]:
!pip install -qU nougat-ocr llama-index langchain

In [4]:
import subprocess
import os

In [5]:
from google.colab import files
upload = files.upload()

Saving mathpaper.pdf to mathpaper.pdf


In [6]:
# Testing the ocr
!nougat --markdown pdf '/content/mathpaper.pdf' --out 'answer'

2023-09-26 21:41:18.725355: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
downloading nougat checkpoint version 0.1.0-small to path /root/.cache/torch/hub/nougat-0.1.0-small
config.json: 100% 557/557 [00:00<00:00, 3.41Mb/s]
pytorch_model.bin: 100% 956M/956M [00:18<00:00, 54.1Mb/s]
special_tokens_map.json: 100% 96.0/96.0 [00:00<00:00, 559kb/s]
tokenizer.json: 100% 2.04M/2.04M [00:00<00:00, 14.8Mb/s]
tokenizer_config.json: 100% 106/106 [00:00<00:00, 725kb/s]
INFO:root:Output directory does not exist. Creating output directory.
/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
  0% 0/10 [00:00<?, ?it/s]WARNING:root:Found repetitions in sample 2
[nltk_data] Downloading package words 

In [7]:
import subprocess

# Define the command as a list of individual arguments
cli_command = [
    'nougat',
    '--markdown',
    'pdf',
    '/content/mathpaper.pdf',
    '--out',
    'output'
]

# Run the command and capture its output
try:
    result = subprocess.run(cli_command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

    # Check the return code to determine success or failure
    if result.returncode == 0:
        print("Command executed successfully")
    else:
        print(f"Command failed with return code {result.returncode}")

    # Print the command's stdout and stderr
    print("Standard Output:")
    print(result.stdout)

    print("Standard Error:")
    print(result.stderr)

except subprocess.CalledProcessError as e:
    print(f"Command failed with return code {e.returncode}: {e.stderr}")


Command executed successfully
Standard Output:

Standard Error:
2023-09-26 21:46:41.047597: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
INFO:root:Output directory does not exist. Creating output directory.
/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]

  0%|          | 0/10 [00:00<?, ?it/s]WARNING:root:Found repetitions in sample 2
INFO:root:Processing file /content/mathpaper.pdf with 37 pages

100%|██████████| 10/10 [04:25<00:00, 26.52s/it]



### Now preparing for LlamaHub loader

In [29]:
import subprocess
import logging
from pathlib import Path
from typing import Dict, Optional, List
from langchain.text_splitter import MarkdownHeaderTextSplitter

from llama_index.readers.base import BaseReader
from llama_index.readers.schema.base import Document

class PDFNougatOCR(BaseReader):
    def nougat_ocr(self, file_path: Path) -> str:
        cli_command = [
            'nougat',
            '--markdown',
            'pdf',
            str(file_path),
            '--out',
            'output'
            '--pages'
        ]

        try:
            result = subprocess.run(
                cli_command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True
            )
            result.check_returncode()
            return result.stdout

        except subprocess.CalledProcessError as e:
            # Log the error message and raise a custom exception
            logging.error(f"Nougat OCR command failed with return code {e.returncode}: {e.stderr}")
            raise RuntimeError("Nougat OCR command failed.") from e

    def load_data(
        self,
        file_path: Path,
        extra_info: Optional[Dict] = None
    ) -> List[Document]:
        try:
            # Ensure the 'output' folder exists or create it if not
            output_folder = Path('output')
            output_folder.mkdir(exist_ok=True)

            # Call the method to run the Nougat OCR command
            output = self.nougat_ocr(file_path)

            # Rest of your code for reading and processing the output
            file_path = Path(file_path)
            output_path = output_folder / f'{file_path.stem}.mmd'
            with output_path.open('r') as f:
                content = f.read()


            content = (
                content.replace(r"\(", "$")
                .replace(r"\)", "$")
                .replace(r"\[", "$$")
                .replace(r"\]", "$$")
            )

            # !!! Need to chunk the before creating Document object here !!!

            return [Document(text=content)]

        except Exception as e:
            logging.error(f"An error occurred while processing the PDF: {str(e)}")

In [30]:
reader = PDFNougatOCR()

data = reader.load_data('/content/mathpaper.pdf')

In [31]:
print(data)


[Document(id_='8c0d2200-4f94-4560-8f41-b431f9c9cbb9', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, hash='25ecf3c54a8d35289dffd13db6f86fe31200d7294ae3048d645c315d7bcf8a98', text='# On ranking _via_ sorting by estimated expected utility\n\n Clement Calauzenes\n\nCriteo AI Lab\n\nParis, France\n\nc.calauzenes@criteo.com &Nicolas Usunier\n\nFacebook AI Research\n\nParis, France\n\nusunier@fb.com\n\n###### Abstract\n\nRanking tasks are defined through losses that measure trade-offs between different desiderata such as the relevance and the diversity of the items at the top of the list. This paper addresses the question of which of these tasks are asymptotically solved by sorting by decreasing order of expected utility, for some suitable notion of utility, or, equivalently, _when is square loss regression consistent for ranking_ via _score-and-sort?_ We answer to this question by finding a characterization of ranking losses fo